In [1]:
#libreries
import pandas as pd
import numpy as np
#to convert the pivot matrix into a matrix
from scipy.sparse import csr_matrix
#NearsetNeighbors is a un-supervised learning. (is not knn)
from sklearn.neighbors import NearestNeighbors

In [2]:
class CollaborativeFiltering:
    def __init__(self, rating_df:str, item_df:str, userId:float, itemId:float,itemTitle:str, rating:float):
        '''Paths of the data sets'''
        #rating data set, that contains the userId, itemId and rating
        self.rating_df = rating_df
        #item data set, that contains the itemId and itemTitle
        self.item_df = item_df

        '''Columns of the data sets'''
        self.userId = userId
        self.itemId = itemId
        self.itemTitle = itemTitle
        self.rating = rating
        
    def import_data(self):
        '''Import data from csv files'''
        self.rating_df = pd.read_csv(self.rating_df, usecols= [self.userId, self.itemId, self.rating])
        self.item_df = pd.read_csv(self.item_df, usecols= [self.itemId, self.itemTitle])
        return self.rating_df, self.item_df
    
    def clean_data(self):
        '''clean data from csv files'''
        #Declaring global df, to use it in other methods
        global df
        #to merge the two data sets
        df = pd.merge(self.rating_df, self.item_df, on = self.itemId)
        #remove the rows with null values
        df. dropna(
            #axis = 0 means removes ROWS that contains NULL values
            axis= 0 , 
            #remove from column itemTitle
            subset= [self.itemTitle],)
        return df
    
    def ratingCount (self, perItem: bool = True, ofTable: bool = False):
        '''This function is to get some statistics from the data'''
        self.perItem = perItem
        self.ofTable = ofTable

        global item_ratingCount
        global df_ratingCount

        #if perItem == True and ofTable == False:
            #number rates per item
        item_ratingCount= (df.
            #group by itemTitle and count the number of ratings
            groupby(by = [self.itemTitle])[self.rating].count().
            #reset the index
            reset_index().
            #rame the col as ratingCount
            rename(columns = {self.rating: 'ratingCount'})
            #show the title and total number of ratings
            [[self.itemTitle, 'ratingCount']]
                            )
            #return item_ratingCount.head()

        self.ofTable == True
        df_ratingCount = df.merge(
            item_ratingCount, 
            left_on= self.itemTitle,
            right_on= self.itemTitle,
            how = 'left')
            #return df_ratingCount.head() 

        if perItem == True and ofTable == False:
            return item_ratingCount.head()
        # perItem == True and ofTable == True:
        else:    
            return df_ratingCount.head()

    def statistics(self):
        '''This function is to get some statistics from the data'''
        return item_ratingCount['ratingCount'].describe()
    
    def create_user_item_matrix(self, popularity = 50):
        '''This function will be based only on popular items'''
        #create a pivot table

        #func()
        rating_popular_item = df_ratingCount.query('ratingCount >= @popularity')
        #return rating_popular_item.shape 
        global item_features_df
        item_features_df = rating_popular_item.pivot_table(
            index = self.itemTitle,
            columns = self.userId,
            values = self.rating).fillna(0)
        return item_features_df

    def knn (self, query_index, k :int):
        '''This function is to get the k nearest neighbors'''
        self.query_index = query_index#[item_features_df.shape[0]]
        self.k = k
        #convert the pivot table into a matrix
        item_features_df_matrix = csr_matrix(item_features_df.values)
        #create a model
        model_knn = NearestNeighbors(metric='cosine', algorithm= 'brute')
        
        #fit the model in the matrix
        model_knn.fit(item_features_df_matrix)

        #distances = what is the distance from the movie Im asking the recomendation for
        #indices = the index information of that particular movie
        distances, indices = model_knn.kneighbors(item_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors = k+1)
       
        # I'm flatten the distances into 1Dimensional array
        # Print recommendations
        print('Recommendations for: \' {0} \': \n'.format(item_features_df.index[query_index]))
        for i in range(1, len(distances.flatten())):  # Start from 1 to skip the query item
            print('{0}: {1}, with distance of {2}:'.format(i,
                                                            item_features_df.index[indices.flatten()[i]],
                                                            distances.flatten()[i]))

In [3]:
cf = CollaborativeFiltering(
    #importing the data sets
    rating_df= '/home/bbruno/all_here/python course/thesis/knn/data/ratings.csv', 
    item_df= '/home/bbruno/all_here/python course/thesis/knn/data/movies.csv',
    #columns to use from the data sets 
    userId= 'userId', itemId= 'movieId', itemTitle= 'title', rating= 'rating')
cf.import_data()

(        userId  movieId  rating
 0            1        1     4.0
 1            1        3     4.0
 2            1        6     4.0
 3            1       47     5.0
 4            1       50     5.0
 ...        ...      ...     ...
 100831     610   166534     4.0
 100832     610   168248     5.0
 100833     610   168250     5.0
 100834     610   168252     5.0
 100835     610   170875     3.0
 
 [100836 rows x 3 columns],
       movieId                                      title
 0           1                           Toy Story (1995)
 1           2                             Jumanji (1995)
 2           3                    Grumpier Old Men (1995)
 3           4                   Waiting to Exhale (1995)
 4           5         Father of the Bride Part II (1995)
 ...       ...                                        ...
 9737   193581  Black Butler: Book of the Atlantic (2017)
 9738   193583               No Game No Life: Zero (2017)
 9739   193585                               Flint (

In [4]:
cf.clean_data()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [5]:
#by default perItem = True and ofTable = False
cf.ratingCount()

#or
#perItem = True and ofTable = True
cf.ratingCount(ofTable= True)

,userId,movieId,rating,title,ratingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [6]:
cf.statistics()

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: ratingCount, dtype: float64

In [7]:
#.get_populars(by default popularity = 50)
cf.create_user_item_matrix()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
You've Got Mail (1998),0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
Young Frankenstein (1974),5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0


In [8]:
cf.knn(query_index= 2, k= 4)

Recommendations for: ' 2001: A Space Odyssey (1968) ': 

1: Blade Runner (1982), with distance of 0.3292643457846739:
2: Alien (1979), with distance of 0.43005304285249724:
3: Apocalypse Now (1979), with distance of 0.4308894589715001:
4: Aliens (1986), with distance of 0.43633477549970023:
